# Uppgift 2

### Antal län i datasetet

In [68]:
import pandas as pd

excel_file_path = "Labb-data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx"

df = pd.read_excel(excel_file_path, sheet_name="Vaccinerade kommun och ålder")

antal_lan = df["Län_namn"].nunique()

print(f"Antal län representerade i datasetet: {antal_lan}")
df

Antal län representerade i datasetet: 21


,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,1,Stockholms län,114,Upplands Väsby,12-15,2422,1206,1046,NaN,NaN,0.497936,0.431874,NaN,NaN
1,1,Stockholms län,114,Upplands Väsby,16-17,1203,839,755,NaN,NaN,0.697423,0.627598,NaN,NaN
2,1,Stockholms län,114,Upplands Väsby,18-29,6692,4887,4469,1959.0,NaN,0.730275,0.667812,0.292738,NaN
3,1,Stockholms län,114,Upplands Väsby,30-39,7332,5542,5240,2878.0,NaN,0.755865,0.714675,0.392526,NaN
4,1,Stockholms län,114,Upplands Väsby,40-49,6946,5592,5429,3719.0,NaN,0.805068,0.781601,0.535416,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,25,Norrbottens län,2584,Kiruna,50-59,3079,2878,2860,2482.0,NaN,0.934719,0.928873,0.806106,NaN
2896,25,Norrbottens län,2584,Kiruna,60-69,2781,2648,2633,2434.0,NaN,0.952175,0.946782,0.875225,NaN
2897,25,Norrbottens län,2584,Kiruna,70-79,2194,2115,2108,2034.0,1784.0,0.963993,0.960802,0.927074,0.813127
2898,25,Norrbottens län,2584,Kiruna,80-89,1280,1256,1253,1220.0,1091.0,0.981250,0.978906,0.953125,0.852344


### Antal kommuner i datsetet 

In [69]:
antal_kommuner = df["Kommun_namn"].nunique()
print(f"Antal kommuner representerade i datasetet: {antal_kommuner}")


Antal kommuner representerade i datasetet: 290


### Antal befolkning

In [70]:
antal_befolkning = df["Befolkning"].sum()
print(f"Antal befolkning som är med i datasetet: {antal_befolkning} personer")

Antal befolkning som är med i datasetet: 9092790 personer


### Antal barn med i datasetet
Antal barn i sverige mellan 12-18 år är runt 779 000, enligt datsetet så är 745 370 av dem vaccinerade (källa: scb)

In [71]:
def extract_lower_bound(age_range):
    if "eller" in age_range:
        return 90  
    return int(age_range.split("-")[0])

df["Lägsta_ålder"] = df["Ålder"].apply(extract_lower_bound)


filtered_df = df[df["Lägsta_ålder"] < 18]

total_population_under_18 = filtered_df["Befolkning"].sum()

print("Antal befolkning under 18:", total_population_under_18)


Antal befolkning under 18: 745370


### Åldersföredling av svergies befolkning

In [72]:
grouped_df = df.groupby("Ålder")["Befolkning"].sum().reset_index()

fig = px.bar(grouped_df, x="Ålder", y="Befolkning", color="Ålder", labels={"Befolkning": "Antal"},
             title="Total befolkning per åldersgrupp")

fig.show()
# fig.write_html("Assigment2e.html")

### Stapeldiagram för Antal vaccinerade i olika län

- To get all län, use .unique and a for-loop to itterate over unique län_namn column.

In [79]:
import pandas as pd
import plotly.express as px

selected_lan = ["Stockholms län", "Västra Götalands län", "Skåne län"]
selected_data = df[df["Län_namn"].isin(selected_lan)]

lan_groups = selected_data.groupby(["Län_namn"]).agg({
    "Befolkning": "sum",
    "Andel minst 1 dos": "mean",  
    "Andel minst 2 doser": "mean",  
    "Andel 3 doser": "mean"  
})

lan_groups = lan_groups.reset_index()

# Multiply by 100 to get percentages
lan_groups["Andel minst 1 dos"] *= 100
lan_groups["Andel minst 2 doser"] *= 100
lan_groups["Andel 3 doser"] *= 100


melted_df = pd.melt(lan_groups, id_vars=["Län_namn"], 
                    value_vars=["Andel minst 1 dos", "Andel minst 2 doser", "Andel 3 doser"],
                    var_name="Dos", value_name="Andel")

fig = px.bar(melted_df,
             x="Län_namn",
             y="Andel",
             color="Dos",
             barmode="group",
             labels={"Andel": "Andel i procent %", "Dos": "Dos", "Län_namn": "Län"},
             title="Andel vaccinerade med dos 1, dos 2 och dos 3 efter län",
             template="plotly_dark"
             )

fig.show()
fig.write_html("Assignment2f.html")


### Jämför vaccinerade i Västra götalands län och Stockholms län

In [76]:
selected_lan = ["Stockholms län", "Västra Götalands län"]
selected_data = df[df["Län_namn"].isin(selected_lan)].copy()  # Create a copy to avoid SettingWithCopyWarning


selected_data["Antal 3 doser"].fillna(0, inplace=True)
selected_data["Antal 4 doser"].fillna(0, inplace=True)


selected_data.loc[:, "Andel minst 1 dos"] = selected_data["Antal minst 1 dos"] / selected_data["Befolkning"] * 100
selected_data.loc[:, "Andel minst 2 doser"] = selected_data["Antal minst 2 doser"] / selected_data["Befolkning"] * 100
selected_data.loc[:, "Andel 3 doser"] = selected_data["Antal 3 doser"] / selected_data["Befolkning"] * 100
selected_data.loc[:, "Andel 4 doser"] = selected_data["Antal 4 doser"] / selected_data["Befolkning"] * 100

lan_groups = selected_data.groupby(["Län_namn"]).agg({
    "Befolkning": "sum",
    "Andel minst 1 dos": "mean",
    "Andel minst 2 doser": "mean",
    "Andel 3 doser": "mean",
    "Andel 4 doser": "mean"
}).reset_index()


melted_lan = pd.melt(lan_groups, id_vars=["Län_namn"], 
                     value_vars=["Andel minst 1 dos", "Andel minst 2 doser", "Andel 3 doser", "Andel 4 doser"],
                     var_name="Dos", value_name="Andel")

fig = px.bar(melted_lan,
             x="Län_namn",
             y="Andel",
             color="Dos",
             barmode="group",
             labels={"Andel": "Andel i procent %", "Dos": "Dos", "Län_namn": "Län"},
             title="Andel vaccinerade med dos 1, dos 2, dos 3 och dos 4 efter län",
             template="plotly_dark"
             )

fig.show()
fig.write_html("Assignment_2g.html")
